In [46]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plot
import numpy as np
import math

In [47]:
team_codes = ['PHI', 'MIL', 'CHI', 'CLE', 'BOS', 'LAC', 'MEM', 'ATL', 'MIA', 'CHO', 'UTA', 'SAC', 'NYK', 
             'LAL', 'ORL', 'DAL', 'BRK', 'DEN', 'IND', 'NOP', 'DET', 'TOR', 'HOU', 'SAS', 'PHO', 'OKC', 
             'MIN', 'POR', 'GSW', 'WAS']
years = [2018, 2019]


NOTE: Using the http request method vs the urllib method doesn't make a big difference in runtime. Currently, http request method is commented out. 

In [48]:
def get_data(year, team, advanced):
    
    url = "https://www.basketball-reference.com/teams/{}/{}/gamelog-advanced/".format(team, year) if advanced else "https://www.basketball-reference.com/teams/{}/{}/gamelog/".format(team, year)
    page = urlopen(url)
    soup = BeautifulSoup(page, 'lxml')
    #page = requests.get(url)
    #soup = BeautifulSoup(page.content, 'lxml')

    headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
    headers = headers[1:]
    if advanced:
        headers[len(headers)-1] = 'dFT/FGA'
        headers[len(headers)-2] = 'dDRB%'
        headers[len(headers)-3] = 'dTOV%'
        headers[len(headers)-4] = 'deFG%'
    else:
        headers[len(headers)-1] = 'dPF'
        headers[len(headers)-9] = 'dFTA'
    rows = soup.findAll('tr')[2:]
    gamelog = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]

    stats = pd.DataFrame(gamelog, columns = headers)
    stats = stats.dropna()
    stats = stats.reset_index(drop=True)

    stats = stats.loc[:,~stats.columns.duplicated()]
    page.close()
    return stats

def combined_data(year, team):
    basic_table = get_data(year, team, 0)
    advanced_table = get_data(year, team, 1)
    temp = pd.concat([basic_table, advanced_table], axis=1, sort=False)
    temp = temp.loc[:,~temp.columns.duplicated()]
    data1 = ['Tm', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST',
       'PF', 'dPF']
    data2 = ['Tm', 'ORtg', 'DRtg', 'Pace', 'FTr',
           '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%',
           'FT/FGA', 'deFG%', 'dTOV%', 'dDRB%', 'dFT/FGA', 'FG', '3P']
    for x in data1:
        temp[x] = pd.to_numeric(temp[x], downcast='float')
    for x in data2:
        temp[x] = pd.to_numeric(temp[x], downcast='float')
    
    temp['2P'] = temp['FG'] - temp['3P']
    temp['2PA'] = temp['FGA'] - temp['3PA']
    temp['2P%'] = temp['2P']/temp['2PA']
    
    return temp



    1. Get the 2 teams that are playing that day
    2. For team A, bootstrap their FT's attempted per game
    3. For team B, bootstrap their FT's allowed per game
    4. FTA for team A will be 0.2*Step2 + 0.8*Step3
    5. Get cumulative FT% for team A and last 5 games FT% for team A and get final FT%
    6. Calculate FT's = Step4*Step5
    7. Boostrap Team A's pace, 3PAr
    8. Obtain 3P% and 2P% for Team A similarly to step 5
    9. Calculate 2p and 3p using formula
    10. Add together FT, 2P, and 3P to get Team A's score 
    11. Repeat steps 2-10 for Team B


In [49]:
# hypothetical game 30 of 2018-2019 season between SAS and ORL
# Team A is SAS, Team B is ORL



def team_fta(teamA_df, teamB_df, teamA_gm_num, teamB_gm_num):
    # Calculating SAS FTA per game
    teamA_fta = np.array(teamA_df['FTA'][0:teamA_gm_num-1], dtype='float')

    # bootstrapping SAS FTA
    means = []
    for x in range(1000):
        temp = np.random.choice(teamA_fta, size=len(teamA_fta))
        means.append(np.mean(temp))
    means = np.array(means)
    teamA_bt_fta = np.mean(means)

    # bootstrapping ORL dFTA
    teamB_dfta = np.array(teamB_df['dFTA'][0:teamB_gm_num-1], dtype='float')
    means = []
    for x in range(1000):
        temp = np.random.choice(teamB_dfta, size=len(teamB_dfta))
        means.append(np.mean(temp))
    means = np.array(means)
    teamB_bt_dfta = np.mean(means)

    # weight the two values
    teamA_total_fta = teamA_bt_fta*0.2 + teamB_bt_dfta*0.8
    
    return teamA_total_fta

In [50]:
def team_ftp(teamA_df, teamA_gm_num):
    # Calculating SAS FT%
    teamA_ft = teamA_df['FT']
    teamA_ft = np.array(teamA_ft, dtype='float')

    teamA_fta = teamA_df['FTA']
    teamA_fta = np.array(teamA_fta, dtype='float')

    teamA_cum_ftp = np.sum(teamA_ft[0:teamA_gm_num-1])/np.sum(teamA_fta[0:teamA_gm_num-1])
    teamA_l5_ftp = np.sum(teamA_ft[teamA_gm_num-6:teamA_gm_num-1])/np.sum(teamA_fta[teamA_gm_num-6:teamA_gm_num-1])

    teamA_true_ftp = teamA_cum_ftp*0.5 + teamA_l5_ftp*0.5
    
    return teamA_true_ftp



In [51]:
# Calculate teamA 2P and 3P
def team_pace(teamA_df, teamA_gm_num): 
    teamA_pace = np.array(teamA_df['Pace'][0:teamA_gm_num-1], dtype='float')

    # bootstrapping teamA Pace
    means = []
    for x in range(1000):
        temp = np.random.choice(teamA_pace, size=len(teamA_pace))
        means.append(np.mean(temp))
    means = np.array(means)
    teamA_bt_pace = np.mean(means)
    
    return teamA_bt_pace


def team_3par(teamA_df, teamA_gm_num):
    teamA_3par = np.array(teamA_df['3PAr'][0:teamA_gm_num-1], dtype='float')

    # bootstrapping teamA 3PAr
    means = []
    for x in range(1000):
        temp = np.random.choice(teamA_3par, size=len(teamA_3par))
        means.append(np.mean(temp))
    means = np.array(means)
    teamA_bt_3par = np.mean(means)
    
    return teamA_bt_3par

def team_2pp(teamA_df, teamA_gm_num):
    # Calculating 2P% for teamA
    teamA_2p = teamA_df['2P']
    teamA_2p = np.array(teamA_2p, dtype='float')

    teamA_2pa = teamA_df['2PA']
    teamA_2pa = np.array(teamA_2pa, dtype='float')

    teamA_cum_2pp = np.sum(teamA_2p[0:teamA_gm_num-1])/np.sum(teamA_2pa[0:teamA_gm_num-1])
    teamA_l5_2pp = np.sum(teamA_2p[teamA_gm_num-6:teamA_gm_num-1])/np.sum(teamA_2pa[teamA_gm_num-6:teamA_gm_num-1])

    teamA_true_2pp = teamA_cum_2pp*0.5 + teamA_l5_2pp*0.5
    
    return teamA_true_2pp

def team_3pp(teamA_df, teamA_gm_num):
    # Calculating 3P% for teamA
    teamA_3p = teamA_df['3P']
    teamA_3p = np.array(teamA_3p, dtype='float')

    teamA_3pa = teamA_df['3PA']
    teamA_3pa = np.array(teamA_3pa, dtype='float')

    teamA_cum_3pp = np.sum(teamA_3p[0:teamA_gm_num-1])/np.sum(teamA_3pa[0:teamA_gm_num-1])
    teamA_l5_3pp = np.sum(teamA_3p[teamA_gm_num-6:teamA_gm_num-1])/np.sum(teamA_3pa[teamA_gm_num-6:teamA_gm_num-1])

    teamA_true_3pp = teamA_cum_3pp*0.5 + teamA_l5_3pp*0.5
    
    return teamA_true_3pp

def calc_team_score(teamA_df, teamB_df, teamA_gm_num, teamB_gm_num):
    # Calculating Total SAS FT for Gm
    teamA_total_ft_pts = team_fta(teamA_df, teamB_df, teamA_gm_num, teamB_gm_num)*team_ftp(teamA_df, teamA_gm_num)
    
    temp_3par = team_3par(teamA_df, teamA_gm_num)
    temp_pace = team_pace(teamA_df, teamA_gm_num)
    teamA_total_2p_pts = temp_pace*(1-temp_3par)*2*team_2pp(teamA_df, teamA_gm_num)
    teamA_total_3p_pts = temp_pace*(temp_3par)*3*team_3pp(teamA_df, teamA_gm_num)

    return teamA_total_ft_pts + teamA_total_2p_pts + teamA_total_3p_pts


In [52]:
xls = pd.ExcelFile('NBAGames.xlsx')
games = pd.read_excel(xls)
spreads = []
for index, row in games.iterrows():
    if math.isnan(row['Predicted Spread']):
        
        teamA_df = combined_data(2021, row['Home Team'])
        teamB_df = combined_data(2021, row['Away Team'])

        teamA_gm_num = teamA_df.shape[0]+1
        teamB_gm_num = teamB_df.shape[0]+1

        teamA_score = calc_team_score(teamA_df, teamB_df, teamA_gm_num, teamB_gm_num)
        teamB_score = calc_team_score(teamB_df, teamA_df, teamB_gm_num, teamA_gm_num)

        predicted = round(teamB_score-teamA_score, 1)
        games.loc[index,'Predicted Spread'] = predicted
        games.loc[index, 'Spread Difference'] = row['Spread'] - predicted

dates = np.array(games['Date'])
temp = []
for x in dates:
    temp.append(pd.to_datetime(x).date())

games['Date'] = temp
# negative spread difference means bet on the away team
games.to_excel('NBAGames.xlsx', index=False )